# Marginal Emissions
## Demonstration on using the marginal emissions feature of NEMED
Using the package <code>nemed</code> to calculate hourly marginal emissions over a historical period.

### Data Preparation
#### Import Packages

In [1]:
import nemed

import pandas as pd
# To generate plots shown
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
# Open plot in browser (optional)
import plotly.io as pio
pio.renderers.default = "browser"

#### Define Parameters

In [3]:
cache = "E:/TEMPCACHE2/"
start_time = "2022/07/01 00:00:00" 
end_time = "2022/08/01 00:00:00"
filter_units = None 
filter_regions = None

#### Get data using NEMED
```{note} 
Extracting 1 month of marginal emissions data was found to take approx. 2 minutes in testing.<br>
Extracting 1 year of marginal emissions data was found to take approx. 40 minutes. Such times may vary depending on computer performance.
```

In [4]:
result = nemed.get_marginal_emissions(start_time, end_time, cache, redownload_xml=True)

Retrieving static table Generators and Scheduled Loads.
No existing JSON found for date 2022-06-29 00:00:00
Redownloading XML data
Converting selected 9505 XML files to JSON, of 178565 cached files


100%|██████████| 9505/9505 [00:54<00:00, 173.85it/s]


Reading JSON to pandas Dataframe
Reading selected 9505 JSON files to pandas, of 178565 cached files


100%|██████████| 9505/9505 [00:13<00:00, 696.77it/s]


#### Aggregate/Filter data
Data retrieve from NEMED can be manually filtered. For example sorting by region we can produce...

In [5]:
# Filter select region
results = {}
for region in ['NSW1','QLD1','VIC1','SA1','TAS1']:

    reg_sum = result[result['RegionID'] == region]
    reg_sum = reg_sum.drop_duplicates(subset=['PeriodID','CO2E_EMISSIONS_FACTOR','tech_name'])
    reg_sum = reg_sum.reset_index(drop=True)[['PeriodID','RegionID','Dispatch Type','tech_name','CO2E_EMISSIONS_FACTOR','Date','Hour']]
    results[region] = reg_sum

In [6]:
results['SA1']

,PeriodID,RegionID,Dispatch Type,tech_name,CO2E_EMISSIONS_FACTOR,Date,Hour
0,2022-07-01 00:05:00,SA1,Generator,Reciprocating Engine,1.491800,2022-07-01,0
1,2022-07-01 00:10:00,SA1,Generator,Hydro - Gravity,0.000000,2022-07-01,0
2,2022-07-01 00:15:00,SA1,Generator,Hydro - Gravity,0.000000,2022-07-01,0
3,2022-07-01 00:20:00,SA1,Generator,Hydro - Gravity,0.000000,2022-07-01,0
4,2022-07-01 00:25:00,SA1,Generator,Black Coal,0.910143,2022-07-01,0
...,...,...,...,...,...,...,...
10473,2022-07-31 23:50:00,SA1,Generator,CCGT,0.477585,2022-07-31,23
10474,2022-07-31 23:55:00,SA1,Generator,Black Coal,0.908305,2022-07-31,23
10475,2022-07-31 23:55:00,SA1,Generator,Black Coal,0.872190,2022-07-31,23
10476,2022-08-01 00:00:00,SA1,Generator,Black Coal,0.908518,2022-08-01,0


Now viewing the hourly marginal emissions intensity by quantiles

In [7]:
perct_regions = {}
for region in ['NSW1','QLD1','VIC1','SA1','TAS1']:
    perct_reg = pd.DataFrame()
    quantiles = [0.4,0.5,0.6,0.7,0.8,0.9,0.95,0.99]
    for q in quantiles:
        subset = results[region]
        q_results = subset.groupby('Hour')['CO2E_EMISSIONS_FACTOR'].quantile(q)
        q_results.name = str(q)
        perct_reg = pd.concat([perct_reg,pd.DataFrame(q_results, columns=[str(q)])],axis=1)
    
    perct_regions[region] = perct_reg


In [8]:
perct_regions['NSW1']

,0.4,0.5,0.6,0.7,0.8,0.9,0.95,0.99
Hour,,,,,,,,
0,0.447535,0.681150,0.908305,0.908518,0.910143,0.910143,0.971761,1.249173
1,0.477585,0.681150,0.908305,0.908518,0.910143,0.971761,0.971761,0.971761
2,0.681150,0.872190,0.908305,0.908305,0.910143,0.910143,0.971761,1.141021
3,0.544686,0.872190,0.908305,0.908305,0.910143,0.912643,0.971761,1.027617
4,0.839124,0.872190,0.908305,0.908518,0.910143,0.971761,0.971761,0.971761
5,0.000000,0.473708,0.872190,0.908305,0.910143,0.910143,0.971761,0.971761
6,0.000000,0.000000,0.000000,0.681150,0.908305,0.910143,0.971761,0.971761
7,0.000000,0.000000,0.000000,0.573436,0.908305,0.910143,0.971761,0.971761
8,0.000000,0.000000,0.573436,0.871459,0.908305,0.910143,0.910143,0.971761


#### Setup Plotting
##### Marginal Emissions Intensity Plot Format
Showing percentiles of marginal emissions intensity on an hourly basis for the defined study period, by region

In [9]:
def plot_mar_em_quantile(data, region):
    fig = go.Figure()
    colors = px.colors.qualitative.Dark2

    fig.update_layout(template="plotly_white", title=f"{region} Marginal Emissions Hourly Intensity - July 2022<br><sub>NEM Emissions Data Tool (NEMED)</sub>",\
        font={'family':'Raleway', 'size': 16}, legend={'title':'Percentile','orientation':'h', 'xanchor': 'center', 'x': 0.5, 'y':-0.2})
    fig.update_yaxes(title_text="Emissions Intensity (tCO2-e/MWh)",range=[-0.1,1.8])
    fig.update_xaxes(title_text="Hour")

    data = data[region]
    for idx, series in enumerate(data.columns):
        fig.add_trace(go.Scatter(x=data.index,y=data[series],\
            name=f"{series}",mode="lines+markers",line_color=colors[idx]))
    fig.show()


def plot_marginal_emissions_intensity(data, color_map={'NSW1':1, 'QLD1':2, 'VIC1':3, 'SA1':4, 'TAS1':5}):

    fig = go.Figure()
    colors = px.colors.qualitative.Dark2

    fig.update_layout(template="plotly_white", title=f"NEM Regions Mean Marginal Emissions Intensity - July 2022<br><sub>NEM Emissions Data Tool (NEMED)</sub>",\
        font={'family':'Raleway', 'size': 12}, legend={'orientation':'h', 'xanchor': 'center', 'x': 0.5, 'y':-0.2})
    fig.update_yaxes(title_text="Mean Emissions Intensity (tCO2-e/MWh)")
    fig.update_xaxes(title_text="Hour")

    for region in results.keys():
        fig.add_trace(go.Scatter(x=results[region]['Hour'],y=results[region]['CO2E_EMISSIONS_FACTOR'],\
            name=f"{region}",mode="lines+markers",line_color=colors[color_map[region]]))

    fig.show()

NSW

In [10]:
plot_mar_em_quantile(perct_regions, 'NSW1')

```{include} charts_ex2/hour_percentiles_nsw.html
```

VIC

In [11]:
plot_mar_em_quantile(perct_regions, 'VIC1')

```{include} charts_ex2/hour_percentiles_vic.html
```

QLD

In [12]:
plot_mar_em_quantile(perct_regions, 'QLD1')

```{include} charts_ex2/hour_percentiles_qld.html
```

SA

In [13]:
plot_mar_em_quantile(perct_regions, 'SA1')

```{include} charts_ex2/hour_percentiles_sa.html
```

TAS

In [14]:
plot_mar_em_quantile(perct_regions, 'TAS1')

```{include} charts_ex2/hour_percentiles_tas.html
```